In [1]:
import os 
os.environ['AWS_PROFILE'] = 'admin'
os.environ['HAVEN_DATABASE'] = 'haven'

import plotly.express as px
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
from random import choice
import h3

from mirrorverse.utils import read_data_w_cache

In [2]:
model = '3_5_1'
run_id = 'a43c72eff441211221256194305afd6a9ceb5a277a01f3853cf2716fa06725e8'
#model = '3_1_4' #'3_1_18' #'3_5_1'
#run_id = '00cf23b296999368ea18b82e33b8687c51e8c35e876afd325e26317cb69ea45b' #'dcdcf74abea8d96ff28901fcd9653fc783df9cdda20ed550adc8c2c38a11d896' #'a43c72eff441211221256194305afd6a9ceb5a277a01f3853cf2716fa06725e8' 
data = read_data_w_cache(
    f"select *, month(time) as month from chinook_depth_inference_{model} where run_id = '{run_id}' and not _train"
)
data['max_depth_bin'] = data.groupby(['_individual', '_decision'])['n_depth_bin'].transform('max')
data['radians'] = np.arctan2(data['sin_sun'], data['cos_sun'])
data['nll'] = -np.log(data['probability'])
print(data.shape)
data.head()

(1285984, 49)


,habitat,_individual,_decision,_choice,_selected,tag_key,h3_index,time,depth_bin,n_depth_bin,...,odds,probability,experiment_name,run_id,_train,_partition,month,max_depth_bin,radians,nll
0,0.958996,73,320036,1,False,159014b,840c53bffffffff,2017-04-01,25.0,0.1,...,14.407755,0.633601,chinook-depth-3-5-1,a43c72eff441211221256194305afd6a9ceb5a277a01f3...,False,0,4,0.4,-0.171004,0.456336
1,0.958996,73,320036,2,False,159014b,840c53bffffffff,2017-04-01,50.0,0.2,...,4.129467,0.181599,chinook-depth-3-5-1,a43c72eff441211221256194305afd6a9ceb5a277a01f3...,False,0,4,0.4,-0.171004,1.705954
2,0.958996,73,320036,4,True,159014b,840c53bffffffff,2017-04-01,100.0,0.4,...,1.852618,0.081471,chinook-depth-3-5-1,a43c72eff441211221256194305afd6a9ceb5a277a01f3...,False,0,4,0.4,-0.171004,2.507502
3,0.958996,73,320036,3,False,159014b,840c53bffffffff,2017-04-01,75.0,0.3,...,2.349631,0.103328,chinook-depth-3-5-1,a43c72eff441211221256194305afd6a9ceb5a277a01f3...,False,0,4,0.4,-0.171004,2.269844
4,0.958996,73,320084,2,False,159014b,840c53bffffffff,2017-03-31,50.0,0.2,...,2.712356,0.144291,chinook-depth-3-5-1,a43c72eff441211221256194305afd6a9ceb5a277a01f3...,False,0,3,0.4,1.753263,1.935926


In [3]:
key_info = {
    'radians': np.pi / 4,
    'month': 1,
    'max_depth_bin': 0.1,
}
key_cols = []
for key, bound in key_info.items():
    data[f'key_{key}'] = round(data[key] / bound).astype(int)
    key_cols.append(f'key_{key}')
data.head()

,habitat,_individual,_decision,_choice,_selected,tag_key,h3_index,time,depth_bin,n_depth_bin,...,run_id,_train,_partition,month,max_depth_bin,radians,nll,key_radians,key_month,key_max_depth_bin
0,0.958996,73,320036,1,False,159014b,840c53bffffffff,2017-04-01,25.0,0.1,...,a43c72eff441211221256194305afd6a9ceb5a277a01f3...,False,0,4,0.4,-0.171004,0.456336,0,4,4
1,0.958996,73,320036,2,False,159014b,840c53bffffffff,2017-04-01,50.0,0.2,...,a43c72eff441211221256194305afd6a9ceb5a277a01f3...,False,0,4,0.4,-0.171004,1.705954,0,4,4
2,0.958996,73,320036,4,True,159014b,840c53bffffffff,2017-04-01,100.0,0.4,...,a43c72eff441211221256194305afd6a9ceb5a277a01f3...,False,0,4,0.4,-0.171004,2.507502,0,4,4
3,0.958996,73,320036,3,False,159014b,840c53bffffffff,2017-04-01,75.0,0.3,...,a43c72eff441211221256194305afd6a9ceb5a277a01f3...,False,0,4,0.4,-0.171004,2.269844,0,4,4
4,0.958996,73,320084,2,False,159014b,840c53bffffffff,2017-03-31,50.0,0.2,...,a43c72eff441211221256194305afd6a9ceb5a277a01f3...,False,0,3,0.4,1.753263,1.935926,2,3,4


In [4]:
model = '3_1_4'
run_id = '00cf23b296999368ea18b82e33b8687c51e8c35e876afd325e26317cb69ea45b'
#model = '3_1_3' #'3_1_4' #'3_1_18'
#run_id = 'e875c3a83c56925e0537b30c6f64d3219ffcd41c2298490d69eec4c25899119c' #'00cf23b296999368ea18b82e33b8687c51e8c35e876afd325e26317cb69ea45b' #'dcdcf74abea8d96ff28901fcd9653fc783df9cdda20ed550adc8c2c38a11d896'
compare = read_data_w_cache(
    f"select *, month(time) as month from chinook_depth_inference_{model} where run_id = '{run_id}' and not _train"
)
compare['max_depth_bin'] = compare.groupby(['_individual', '_decision'])['n_depth_bin'].transform('max')
compare['radians'] = np.arctan2(compare['sin_sun'], compare['cos_sun'])
compare['nll'] = -np.log(compare['probability'])
print(compare.shape)
compare.head()

(1285984, 48)


,_individual,_decision,_choice,_selected,tag_key,h3_index,time,depth_bin,n_depth_bin,cos_moon,...,odds,probability,experiment_name,run_id,_train,_partition,month,max_depth_bin,radians,nll
0,73,318592,1,True,159014b,840c503ffffffff,2017-04-26,25.0,0.1,0.994033,...,0.021925,0.730627,chinook-depth-3-1-4,00cf23b296999368ea18b82e33b8687c51e8c35e876afd...,False,0,4,0.2,-0.594858,0.313852
1,73,318592,2,False,159014b,840c503ffffffff,2017-04-26,50.0,0.2,0.994033,...,0.008084,0.269373,chinook-depth-3-1-4,00cf23b296999368ea18b82e33b8687c51e8c35e876afd...,False,0,4,0.2,-0.594858,1.311659
2,73,318608,1,True,159014b,840c539ffffffff,2017-05-08,25.0,0.1,-0.778293,...,0.046759,0.726677,chinook-depth-3-1-4,00cf23b296999368ea18b82e33b8687c51e8c35e876afd...,False,0,5,0.3,1.117011,0.319274
3,73,318608,3,False,159014b,840c539ffffffff,2017-05-08,75.0,0.3,-0.778293,...,0.006830,0.106146,chinook-depth-3-1-4,00cf23b296999368ea18b82e33b8687c51e8c35e876afd...,False,0,5,0.3,1.117011,2.242940
4,73,318608,2,False,159014b,840c539ffffffff,2017-05-08,50.0,0.2,-0.778293,...,0.010757,0.167177,chinook-depth-3-1-4,00cf23b296999368ea18b82e33b8687c51e8c35e876afd...,False,0,5,0.3,1.117011,1.788700


In [5]:
key_info = {
    'radians': np.pi / 4,
    'month': 1,
    'max_depth_bin': 0.1,
}
key_cols = []
for key, bound in key_info.items():
    compare[f'key_{key}'] = round(compare[key] / bound).astype(int)
    key_cols.append(f'key_{key}')
compare.head()

,_individual,_decision,_choice,_selected,tag_key,h3_index,time,depth_bin,n_depth_bin,cos_moon,...,run_id,_train,_partition,month,max_depth_bin,radians,nll,key_radians,key_month,key_max_depth_bin
0,73,318592,1,True,159014b,840c503ffffffff,2017-04-26,25.0,0.1,0.994033,...,00cf23b296999368ea18b82e33b8687c51e8c35e876afd...,False,0,4,0.2,-0.594858,0.313852,-1,4,2
1,73,318592,2,False,159014b,840c503ffffffff,2017-04-26,50.0,0.2,0.994033,...,00cf23b296999368ea18b82e33b8687c51e8c35e876afd...,False,0,4,0.2,-0.594858,1.311659,-1,4,2
2,73,318608,1,True,159014b,840c539ffffffff,2017-05-08,25.0,0.1,-0.778293,...,00cf23b296999368ea18b82e33b8687c51e8c35e876afd...,False,0,5,0.3,1.117011,0.319274,1,5,3
3,73,318608,3,False,159014b,840c539ffffffff,2017-05-08,75.0,0.3,-0.778293,...,00cf23b296999368ea18b82e33b8687c51e8c35e876afd...,False,0,5,0.3,1.117011,2.242940,1,5,3
4,73,318608,2,False,159014b,840c539ffffffff,2017-05-08,50.0,0.2,-0.778293,...,00cf23b296999368ea18b82e33b8687c51e8c35e876afd...,False,0,5,0.3,1.117011,1.788700,1,5,3


In [6]:
bdf = data[data['_selected']].groupby(key_cols).agg({'nll': 'mean', '_choice': 'count'}).rename(columns={'_choice': 'samples'}).reset_index()
cdf = compare[compare['_selected']].groupby(key_cols).agg({'nll': 'mean'}).reset_index()
df = bdf.merge(cdf, on=key_cols, suffixes=('_habitat', ''))
df['nll_diff'] = df['nll'] - df['nll_habitat']
df = df.sort_values('nll_diff', ascending=False)
df['bin_proportion'] = 1.0
df['bin_proportion'] = df['bin_proportion'].cumsum() / df['bin_proportion'].sum()
df['proportion'] = df['samples'].cumsum() / df['samples'].sum()
df.head()

,key_radians,key_month,key_max_depth_bin,nll_habitat,samples,nll,nll_diff,bin_proportion,proportion
60,-4,8,9,2.726047,9,3.467427,0.741380,0.001196,0.000047
803,4,8,9,2.841084,15,3.506855,0.665771,0.002392,0.000125
165,-3,10,7,1.021592,152,1.448107,0.426516,0.003589,0.000914
710,3,8,9,3.038245,31,3.426933,0.388688,0.004785,0.001075
258,-2,10,7,1.213420,71,1.582152,0.368731,0.005981,0.001443


In [12]:
counts = data.groupby(key_cols)['_individual'].nunique().reset_index().rename(columns={'_individual': 'unique_individuals'})
px.box(
    df.merge(counts), x='unique_individuals', y='nll_diff'
)

In [11]:
counts = data.groupby(key_cols)['_individual'].nunique().reset_index().rename(columns={'_individual': 'unique_individuals'})
px.line(
    df.merge(counts[counts['unique_individuals'] >= 5][key_cols]), x='nll_diff', y='proportion'
)

In [8]:
px.line(
    df, x='nll_diff', y='bin_proportion'
)

In [15]:
counts = data.groupby(key_cols)['_individual'].nunique().reset_index().rename(columns={'_individual': 'unique_individuals'})
data = data[[c for c in data.columns if c != 'unique_individuals']].merge(counts)
compare = compare[[c for c in compare.columns if c != 'unique_individuals']].merge(counts)

In [ ]:
bdf = data[data['_selected']].groupby('unique_individuals').agg({'nll': 'mean', '_choice': 'count'}).rename(columns={'_choice': 'proportion'})
bdf['proportion'] = round(bdf['proportion'] / bdf['proportion'].sum(), 2)
cdf = compare[compare['_selected']].groupby('unique_individuals')['nll'].mean().reset_index()
bdf.merge(cdf, on='unique_individuals', suffixes=('_w_habitat', ''))

,unique_individuals,nll_w_habitat,proportion,nll
0,1,1.634324,0.03,1.632974
1,2,1.358455,0.09,1.390419
2,3,1.185446,0.17,1.186816
3,4,1.346740,0.17,1.379824
4,5,1.297012,0.06,1.293017
5,6,1.539854,0.12,1.549070
6,7,1.435231,0.16,1.443237
7,8,1.573355,0.12,1.601032
8,9,1.564556,0.04,1.627464
9,10,1.698459,0.04,1.713915


: 

In [4]:
def get_likelihoods(x):
    likelihoods = {}
    for _, row in x.iterrows():
        likelihoods[row['depth_bin']] = row['probability']
    return likelihoods

odf = data.groupby(['_individual', '_decision'] + key_cols).apply(get_likelihoods).reset_index().rename(columns={0: 'likelihoods'})
odf

/tmp/ipykernel_69035/1204984729.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  odf = data.groupby(['_individual', '_decision'] + key_cols).apply(get_likelihoods).reset_index().rename(columns={0: 'likelihoods'})


,_individual,_decision,key_radians,key_month,key_max_depth_bin,likelihoods
0,73,318534,-3,5,3,"{25.0: 0.702519953250885, 50.0: 0.191676750779..."
1,73,318535,-3,5,3,"{50.0: 0.17820896208286285, 75.0: 0.1004687473..."
2,73,318536,2,5,3,"{75.0: 0.07992267608642578, 50.0: 0.1683628708..."
3,73,318537,-4,5,3,"{25.0: 0.7258635759353638, 50.0: 0.17636090517..."
4,73,318538,4,5,3,"{50.0: 0.17890435457229614, 25.0: 0.7279314398..."
...,...,...,...,...,...,...
192638,110,521549,-2,8,7,"{50.0: 0.14208410680294037, 150.0: 0.014608704..."
192639,110,521550,-2,8,7,"{25.0: 0.7524167895317078, 50.0: 0.14140169322..."
192640,110,521551,-2,8,7,"{250.0: 0.0012347715673968196, 25.0: 0.7248201..."
192641,110,521552,1,8,7,"{150.0: 0.07553541660308838, 250.0: 0.00935001..."


In [5]:
likelihoods = defaultdict(list)
for _, row in tqdm(odf.iterrows(), total=odf.shape[0]):
    key = tuple(row[key] for key in key_cols)
    likelihoods[key].append(row['likelihoods'])

100%|██████████| 192643/192643 [00:04<00:00, 43289.24it/s]


In [6]:
decisions = data[['_individual', '_decision'] + key_cols].drop_duplicates()
permutations = []
for i in range(1):
    print(f"Permutation {i}")
    rows = []
    for _, row in tqdm(decisions.iterrows(), total=decisions.shape[0]):
        key = tuple(row[key] for key in key_cols)
        likelihood = choice(likelihoods[key])
        for depth_bin, prob in likelihood.items():
            rows.append({
                '_individual': row['_individual'],
                '_decision': row['_decision'],
                'depth_bin': depth_bin,
                'probability': prob,
                **{key: row[key] for key in key_cols},
            })
    perm = pd.DataFrame(rows).merge(data[['_individual', '_decision', 'depth_bin', '_selected', 'salinity', 'mixed_layer_thickness', 'nitrate']], on=['_individual', '_decision', 'depth_bin'], how='inner')
    perm['permutation'] = i
    permutations.append(perm)
perm = pd.concat(permutations, ignore_index=True)
print(perm.shape)
perm.head()

Permutation 0


100%|██████████| 192643/192643 [00:14<00:00, 13143.29it/s]


(1285984, 12)


,_individual,_decision,depth_bin,probability,key_radians,key_month,key_max_depth_bin,_selected,salinity,mixed_layer_thickness,nitrate,permutation
0,73,320036,75.0,0.088068,0,4,4,False,32.047225,94.016088,6.757948,0
1,73,320036,25.0,0.691271,0,4,4,False,32.031617,88.726031,6.654285,0
2,73,320036,100.0,0.065896,0,4,4,True,32.052041,94.016088,6.746055,0
3,73,320036,50.0,0.154765,0,4,4,False,32.039954,91.048387,6.740292,0
4,73,320084,100.0,0.044027,2,3,4,False,32.030725,98.355439,6.741712,0


In [7]:
perm['nll'] = -np.log(perm['probability'])
data['nll'] = -np.log(data['probability'])
perm[perm['_selected']]['nll'].mean(), data[data['_selected']]['nll'].mean()

(1.428741812673331, 1.414138)

In [8]:
pdf = perm[perm['_selected']].groupby(key_cols)['nll'].mean().reset_index()
adf = data[data['_selected']].groupby(key_cols)['nll'].mean().reset_index()
df = pdf.merge(adf, on=key_cols, how='inner', suffixes=('_perm', '_data'))
df['nll_diff'] = df['nll_perm'] - df['nll_data']
df.sort_values('nll_diff', ascending=False, inplace=True)
df

,key_radians,key_month,key_max_depth_bin,nll_perm,nll_data,nll_diff
425,0,7,9,2.560318,2.229744,0.330574
164,-3,10,6,1.747396,1.528741,0.218655
584,2,4,6,1.638764,1.423019,0.215744
492,1,4,6,1.589701,1.377871,0.211830
399,0,4,6,1.596595,1.385257,0.211338
...,...,...,...,...,...,...
15,-4,3,3,0.943617,1.029711,-0.086094
49,-4,7,6,1.643700,1.734859,-0.091159
795,4,7,9,2.590332,2.692478,-0.102146
758,4,3,3,0.856470,0.973017,-0.116546


In [35]:
cdf = df.copy()
cdf['bins'] = 1
cdf = cdf.sort_values('nll_diff', ascending=False)
cdf['bins'] = cdf['bins'].cumsum()
cdf['bin_proportion'] = cdf['bins'] / cdf['bins'].max()

px.line(cdf, x='nll_diff', y='bin_proportion')

In [9]:
quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]
sdf = pd.DataFrame({
    'quantile': quantiles,
    'nll_diff': [df['nll_diff'].quantile(q) for q in quantiles],
})
sdf

,quantile,nll_diff
0,0.10,-0.020030
1,0.25,-0.005669
2,0.50,0.003682
3,0.75,0.018236
4,0.90,0.043612


In [21]:
cdf = df.merge(data.groupby(key_cols).size().reset_index().rename(columns={0: 'count'})).sort_values('nll_diff', ascending=False)
cdf['cumsum'] = cdf['count'].cumsum()
cdf['proportion'] = cdf['cumsum'] / cdf['count'].sum()
cdf.head()

,key_radians,key_month,key_max_depth_bin,nll_perm,nll_data,nll_diff,count,cumsum,proportion
0,0,7,9,2.560318,2.229744,0.330574,108,108,0.000084
1,-3,10,6,1.747396,1.528741,0.218655,2478,2586,0.002011
2,2,4,6,1.638764,1.423019,0.215744,2550,5136,0.003994
3,1,4,6,1.589701,1.377871,0.211830,2652,7788,0.006056
4,0,4,6,1.596595,1.385257,0.211338,2334,10122,0.007871


In [22]:
px.line(
    cdf, x='nll_diff', y='proportion'
)

In [37]:
_filter = data.groupby(key_cols)['_individual'].nunique().reset_index().sort_values('_individual', ascending=True)
_filter = _filter[_filter['_individual'] >= 5]
fdf = df.merge(_filter[key_cols])
fdf

,key_radians,key_month,key_max_depth_bin,nll_perm,nll_data,nll_diff,bins,bin_proportion
0,-3,10,6,1.747396,1.528741,0.218655,2,0.002392
1,-1,10,6,1.646114,1.475067,0.171047,6,0.007177
2,-4,10,6,1.803614,1.645337,0.158277,10,0.011962
3,0,10,6,1.842163,1.719628,0.122535,15,0.017943
4,-2,10,6,1.677292,1.555156,0.122135,16,0.019139
...,...,...,...,...,...,...,...,...
192,1,11,7,1.852999,1.878648,-0.025649,785,0.938995
193,4,12,6,1.803291,1.835441,-0.032149,792,0.947368
194,2,10,4,1.335934,1.370613,-0.034679,798,0.954545
195,-4,3,6,1.715026,1.754421,-0.039395,807,0.965311


In [38]:
cdf = df.merge(_filter[key_cols]).copy()
cdf['bins'] = 1
cdf = cdf.sort_values('nll_diff', ascending=False)
cdf['bins'] = cdf['bins'].cumsum()
cdf['bin_proportion'] = cdf['bins'] / cdf['bins'].max()

px.line(cdf, x='nll_diff', y='bin_proportion')

In [39]:
cdf = df.merge(_filter[key_cols]).copy()
cdf['bins'] = 1
cdf = cdf.sort_values('nll_diff', ascending=True)
cdf['bins'] = cdf['bins'].cumsum()
cdf['bin_proportion'] = cdf['bins'] / cdf['bins'].max()

px.line(cdf, x='nll_diff', y='bin_proportion')

In [32]:
cdf = df.merge(_filter[key_cols]).merge(data.groupby(key_cols).size().reset_index().rename(columns={0: 'count'})).sort_values('nll_diff', ascending=False)
cdf['cumsum'] = cdf['count'].cumsum()
cdf['proportion'] = cdf['cumsum'] / cdf['count'].sum()
cdf.head()

,key_radians,key_month,key_max_depth_bin,nll_perm,nll_data,nll_diff,count,cumsum,proportion
0,-3,10,6,1.747396,1.528741,0.218655,2478,2478,0.003496
1,-1,10,6,1.646114,1.475067,0.171047,2010,4488,0.006332
2,-4,10,6,1.803614,1.645337,0.158277,1224,5712,0.008059
3,0,10,6,1.842163,1.719628,0.122535,2052,7764,0.010954
4,-2,10,6,1.677292,1.555156,0.122135,2238,10002,0.014112


In [33]:
px.line(
    cdf, x='nll_diff', y='proportion'
)

In [ ]:
fdf[fdf['nll_diff'] < -0.05].shape[0] / df.shape[0]

In [ ]:
data.merge(
    df[df['nll_diff'] < -0.05][['key_radians', 'key_month', 'key_max_depth_bin']], 
    how='inner'
).shape[0] / data.shape[0]

In [ ]:
df[(df['key_month'] == 1) & (df['key_max_depth_bin'].isin([4]))].groupby(['key_sin_sun'])['nll_diff'].describe()

In [ ]:
pdf = perm[(perm['key_max_depth_bin'] == 7) & (perm['key_month'] == 11) & (perm['key_sin_sun'] >= 0)]
adf = data[(data['key_max_depth_bin'] == 7) & (data['key_month'] == 11) & (data['key_sin_sun'] >= 0)]
df = pdf.merge(adf[['_individual', '_decision', 'depth_bin', 'nll']], on=['_individual', '_decision', 'depth_bin'], how='inner', suffixes=('_perm', '_data'))
df['nll_diff'] = df['nll_perm'] - df['nll_data']
df['max_salinity'] = df.groupby(['_individual', '_decision'])['salinity'].transform('max')
df['min_salinity'] = df.groupby(['_individual', '_decision'])['salinity'].transform('min')
df['salinity_diff'] = df['max_salinity'] - df['min_salinity']
df['binned_salinity_diff'] = round(df['salinity_diff'] / 0.5) * 0.5
df['binned_salinity'] = round(df['salinity'] / 0.5) * 0.5
px.scatter(
    df[df['_selected']].groupby(['binned_salinity'])['nll_diff'].mean().reset_index(),
    x='binned_salinity', y='nll_diff',
)

In [ ]:
adf = data[(data['key_max_depth_bin'] == 7) & (data['key_month'] == 11) & (data['key_sin_sun'] >= 0)]
adf['max_salinity'] = adf.groupby(['_individual', '_decision'])['salinity'].transform('max')
adf['min_salinity'] = adf.groupby(['_individual', '_decision'])['salinity'].transform('min')
adf['stratified'] = (adf['max_salinity'] - adf['min_salinity']) >= 1
adf[adf['_selected'] & (adf['stratified'])].groupby(['depth_bin']).size()

In [ ]:
adf[adf['_selected'] & (~adf['stratified'])].groupby(['depth_bin']).size()

In [ ]:
adf = data[(data['key_max_depth_bin'] > 4) & (data['key_month'] == 8) & (data['key_sin_sun'] >= 0)]
adf['max_salinity'] = adf.groupby(['_individual', '_decision'])['salinity'].transform('max')
adf['min_salinity'] = adf.groupby(['_individual', '_decision'])['salinity'].transform('min')
adf['saline'] = adf['min_salinity'] >= 32

saline = pd.DataFrame((adf[adf['_selected'] & (adf['saline'])].groupby(['depth_bin']).size() / adf[adf['_selected'] & (adf['saline'])].shape[0])).rename(columns={0: 'proportion'}).reset_index()
saline['proportion'] = round(saline['proportion'], 2)
fresh = pd.DataFrame((adf[adf['_selected'] & (~adf['saline'])].groupby(['depth_bin']).size() / adf[adf['_selected'] & (~adf['saline'])].shape[0])).rename(columns={0: 'proportion'}).reset_index()
fresh['proportion'] = round(fresh['proportion'], 2)
saline.merge(fresh, on='depth_bin', how='inner', suffixes=('_saline', '_fresh'))

In [ ]:
perm[perm['_selected'] & (perm['key_sin_sun'] <= 0) & (perm['salinity'] <= 32)]['nll'].mean(), data[data['_selected'] & (data['key_sin_sun'] <= 0) & (data['salinity'] <= 32)]['nll'].mean()

In [ ]:
(
    perm[perm['_selected'] & (perm['key_sin_sun'] >= 0) & (perm['salinity'] > 32) & (perm['key_max_depth_bin'] > 6) & (perm['key_month'] == 11)]['nll'].mean(), 
    data[data['_selected'] & (data['key_sin_sun'] >= 0) & (data['salinity'] > 32) & (data['key_max_depth_bin'] > 6) & (perm['key_month'] == 11)]['nll'].mean()
)

In [ ]:
df = data[(data['depth_bin'] == 25.0) & (data['key_max_depth_bin'].isin([6, 7, 8])) & (data['key_sin_sun'] >= 0)]
df['min_salinity'] = df.groupby(['_individual', '_decision'])['salinity'].transform('min')
df['binned_min_salinity'] = round(df['min_salinity'] / 0.25) * 0.25
px.scatter(
    df.groupby(['month', 'binned_min_salinity'])['_selected'].agg(['mean', 'count']).reset_index(),
    x='binned_min_salinity', y='mean',  size='count', facet_col='month', category_orders={'month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]},
    facet_col_wrap=4
)

In [ ]:
data['max_salinity'] = data.groupby(['_individual', '_decision'])['salinity'].transform('max')
data['min_salinity'] = data.groupby(['_individual', '_decision'])['salinity'].transform('min')
df = data[(data['depth_bin'] == 25.0) & (data['key_max_depth_bin'].isin([6, 7, 7]))]
df['radians'] = np.arctan2(df['sin_sun'], df['cos_sun'])
df['radians'] = round(df['radians'] * 5) / 5
df['salinity_diff'] = df['max_salinity'] - df['min_salinity']
df['stratified'] = df['salinity_diff'] >= 1
df = df.groupby(['radians', 'stratified', 'month'])['_selected'].agg(['mean', 'count']).reset_index()
px.scatter(
    df, x='radians', y='mean', color='stratified',
    category_orders={'stratified': [True, False], 'month': list(range(1, 13))},
    color_discrete_sequence=['aquamarine', 'navy'],
    facet_col='month', facet_col_wrap=4,
    height=900, width=1000
)

In [ ]:
df = data[data['depth_bin'] == 25.0]
df.columns

In [ ]:
adf[adf['_selected'] & (~adf['saline'])].groupby(['depth_bin']).size() / adf[adf['_selected'] & (~adf['saline'])].shape[0]

In [ ]:
data[data['depth_bin'] == 100]['salinity'].describe()

In [ ]:
space = data[['h3_index', 'key_max_depth_bin']].drop_duplicates()
space['lat'] = space['h3_index'].apply(lambda x: h3.h3_to_geo(x)[0])
space['lon'] = space['h3_index'].apply(lambda x: h3.h3_to_geo(x)[1])
space['size'] = 0.01
fig = px.scatter_mapbox(
    space[space['key_max_depth_bin'].isin([6, 7])],
    lat='lat',
    lon='lon',
    color='key_max_depth_bin',  # Color points by probability
    size='size',  # Adjust as needed
    size_max=11,  # Adjust as needed
    zoom=4,  # Adjust zoom level
    mapbox_style="carto-positron",  # Choose a map style,
    #range_color=[0.0, 0.3],
    height=600,
    width=1000
)
fig.show()